<a href="https://colab.research.google.com/github/Prishi99/WomenLine-AI-/blob/main/Week_3_Language_Handler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langdetect googletrans==4.0.0-rc1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=5d63eecabbab96fa9d3881f15268199871704395dc354a85df2d7e0c1fa43fba
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
  Created wheel for

In [2]:
import json
from langdetect import detect
from googletrans import Translator

# Load English dataset
with open('legal_dataset.json', 'r', encoding='utf-8') as f:
    english_dataset = json.load(f)

# Load Translated Datasets (Optional)
try:
    with open('legal_qa_dataset_hi.json', 'r', encoding='utf-8') as f:
        hindi_dataset = json.load(f)
except FileNotFoundError:
    hindi_dataset = []

try:
    with open('legal_qa_dataset_ta.json', 'r', encoding='utf-8') as f:
        tamil_dataset = json.load(f)
except FileNotFoundError:
    tamil_dataset = []

translator = Translator()

def find_answer_from_dataset(dataset, question):
    for entry in dataset:
        if entry['question'].strip().lower() == question.strip().lower():
            return entry['answer']
    return None

def handle_query(user_input):
    # Detect language
    detected_lang = detect(user_input)
    print(f"Detected language: {detected_lang}")

    # If English, search directly
    if detected_lang == 'en':
        return find_answer_from_dataset(english_dataset, user_input)

    # Use translated dataset if available
    if detected_lang == 'hi':
        answer = find_answer_from_dataset(hindi_dataset, user_input)
        if answer:
            return answer
    elif detected_lang == 'ta':
        answer = find_answer_from_dataset(tamil_dataset, user_input)
        if answer:
            return answer

    # Fallback: Translate to English, search, then translate answer back
    translated_question = translator.translate(user_input, src=detected_lang, dest='en').text
    print(f"Translated question to English: {translated_question}")

    english_answer = find_answer_from_dataset(english_dataset, translated_question)

    if english_answer:
        translated_answer = translator.translate(english_answer, src='en', dest=detected_lang).text
        return translated_answer

    return "Sorry, I couldn't find an answer to your question."

# Example usage
if __name__ == "__main__":
    while True:
        user_q = input("\nAsk a legal question (type 'exit' to stop): ")
        if user_q.lower() == 'exit':
            break
        print("Answer:", handle_query(user_q))



Ask a legal question (type 'exit' to stop): My husband beats me 
Detected language: en
Answer: None

Ask a legal question (type 'exit' to stop): exit


In [3]:
def translate_dataset(dataset, target_lang, output_file):
    translated = []
    for entry in dataset:
        translated.append({
            "intent": entry["intent"],
            "question": translator.translate(entry["question"], src='en', dest=target_lang).text,
            "answer": translator.translate(entry["answer"], src='en', dest=target_lang).text
        })

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(translated, f, ensure_ascii=False, indent=2)

# Usage
# translate_dataset(english_dataset, 'hi', 'legal_qa_dataset_hi.json')
# translate_dataset(english_dataset, 'ta', 'legal_qa_dataset_ta.json')


In [4]:
import json
import time
from googletrans import Translator

# Initialize the Google Translator
translator = Translator()

# Supported languages you want to translate to
languages = {
    'hi': 'Hindi',
    'ta': 'Tamil',
    'bn': 'Bengali',
    'mr': 'Marathi'
}

# Load original English dataset
with open('legal_dataset.json', 'r', encoding='utf-8') as f:
    english_dataset = json.load(f)

def translate_dataset(dataset, target_lang_code):
    translated = []
    for entry in dataset:
        try:
            translated_q = translator.translate(entry['question'], src='en', dest=target_lang_code).text
            translated_a = translator.translate(entry['answer'], src='en', dest=target_lang_code).text

            translated.append({
                'intent': entry['intent'],
                'question': translated_q,
                'answer': translated_a
            })

            # Respect API limits
            time.sleep(0.5)

        except Exception as e:
            print(f"Error translating: {entry['question']}\n{e}\n")
            continue

    return translated

# Translate and save to separate JSON files
for lang_code, lang_name in languages.items():
    print(f"\nTranslating to {lang_name} ({lang_code})...")
    translated_dataset = translate_dataset(english_dataset, lang_code)

    output_file = f'legal_qa_dataset_{lang_code}.json'
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(translated_dataset, f, ensure_ascii=False, indent=2)

    print(f"✅ Saved translated dataset to {output_file}")



Translating to Hindi (hi)...
Error translating: Does the cybercrime portal have a mobile-friendly or app version?
The read operation timed out

✅ Saved translated dataset to legal_qa_dataset_hi.json

Translating to Tamil (ta)...
✅ Saved translated dataset to legal_qa_dataset_ta.json

Translating to Bengali (bn)...
✅ Saved translated dataset to legal_qa_dataset_bn.json

Translating to Marathi (mr)...
✅ Saved translated dataset to legal_qa_dataset_mr.json


In [5]:
import json
import os
from langdetect import detect
from googletrans import Translator

# Load translator
translator = Translator()

# Load all datasets
DATASETS = {}
LANGUAGES = ['en', 'hi', 'ta', 'bn', 'mr']

# Dataset file paths
DATASET_FILES = {
    'en': 'legal_dataset.json',
    'hi': 'legal_qa_dataset_hi.json',
    'ta': 'legal_qa_dataset_ta.json',
    'bn': 'legal_qa_dataset_bn.json',
    'mr': 'legal_qa_dataset_mr.json'
}

# Load datasets into memory
for lang_code, file_name in DATASET_FILES.items():
    if os.path.exists(file_name):
        with open(file_name, 'r', encoding='utf-8') as f:
            DATASETS[lang_code] = json.load(f)
    else:
        DATASETS[lang_code] = []

# Match question by fuzzy normalized string
def find_answer(dataset, user_question):
    question_lower = user_question.strip().lower()
    for entry in dataset:
        if entry['question'].strip().lower() == question_lower:
            return entry['answer']
    return None

def handle_query(user_input):
    # Detect user input language
    try:
        detected_lang = detect(user_input)
    except:
        return "Sorry, couldn't detect your language."

    print(f"Detected language: {detected_lang}")

    # Use available dataset
    if detected_lang in DATASETS:
        answer = find_answer(DATASETS[detected_lang], user_input)
        if answer:
            return answer

    # Fallback: Translate question to English and get answer
    try:
        translated_question = translator.translate(user_input, src=detected_lang, dest='en').text
        english_answer = find_answer(DATASETS['en'], translated_question)
        if not english_answer:
            return "Sorry, no answer found for your question."

        # Translate back to original language
        translated_answer = translator.translate(english_answer, src='en', dest=detected_lang).text
        return translated_answer
    except Exception as e:
        print("Translation error:", e)
        return "Sorry, something went wrong with translation."

# Demo / Test loop
if __name__ == "__main__":
    print("🔍 Multilingual Legal QA System (Type 'exit' to quit)")
    while True:
        user_q = input("\nAsk your question: ")
        if user_q.lower() == 'exit':
            break
        response = handle_query(user_q)
        print("Answer:", response)


🔍 Multilingual Legal QA System (Type 'exit' to quit)

Ask your question: भारत में घरेलू दुर्व्यवहार क्या है?
Detected language: hi
Answer: घरेलू दुर्व्यवहार में घरेलू संबंधों के भीतर शारीरिक, भावनात्मक, यौन, मौखिक और आर्थिक दुर्व्यवहार शामिल हैं।[स्रोत: घरेलू हिंसा अधिनियम, 2005 से महिलाओं की सुरक्षा]

Ask your question: भारत में घरेलू दुर्व्यवहार क्या है?
Detected language: hi
Answer: घरेलू दुर्व्यवहार में घरेलू संबंधों के भीतर शारीरिक, भावनात्मक, यौन, मौखिक और आर्थिक दुर्व्यवहार शामिल हैं।[स्रोत: घरेलू हिंसा अधिनियम, 2005 से महिलाओं की सुरक्षा]

Ask your question: exit
